In [9]:
import cvxpy as cp 
import numpy as np
import pandas as pd
import os



In [5]:
#df = pd.read_excel('/Users/emariedelanuez/Downloads/clean_data.xlsx')
#data = np.array(df)

In [19]:
#np.set_printoptions(threshold=np.inf, linewidth=np.inf)


In [37]:
excel_file = pd.read_excel('/Users/emariedelanuez/Downloads/clean_data.xlsx', engine='openpyxl', sheet_name=None)
# iterate through dictionary and create separate data frames
for sheet_name, sheet_data in excel_file.items():
    # create separate data frame for each sheet
    df = pd.DataFrame(sheet_data)
    # save data frame as CSV file
    df.to_csv(f'{sheet_name}.csv', index=False)


In [34]:
# Get the current working directory
cwd = os.getcwd()
#print(f"Current working directory: {cwd}")

# List the files in the directory
files = os.listdir(cwd)
#print(f"Files in the directory: {files}")

In [39]:
#reading in files
model = pd.read_csv('model.csv')
target1 = pd.read_csv('target1.csv')
target2 = pd.read_csv('target2.csv')
target3 = pd.read_csv('target3.csv')


In [145]:
#making model c_dts just to make sure
model['constant'] = model['oas'] * model['oasd']* model['weight']
model.dropna(subset=['constant'], inplace=True)
#print(model['constant'])

1      1.820536
3      0.803512
4      1.845752
5      1.527298
9      0.731480
         ...   
556    2.353128
557    1.332228
559    0.297969
560    0.448484
561    0.244891
Name: constant, Length: 238, dtype: float64


In [157]:
##making target1 d_t vector
target1['d_t'] = target1['oas'] * target1['oasd']
target1.dropna(subset=['d_t'], inplace=True)
#print(target1['d_t'])





#transposed_d_t_array = d_t_array.transpose()
#transposed_d_t = target1[['d_t']].T
##convert the transposed column to a dataframe
#transposed_dt = pd.DataFrame(transposed_d_t, columns=['transposed_d_t'])
# set the display option to show all rows
#pd.set_option('display.max_rows', 10)

##print the transposed dataframe
#print(transposed_dt)
#pd.set_option('display.max_rows',10)
# print the transposed column
#print(target1['transposed_d_t'])
#print(target1)




In [147]:
###pandas into numpy array
constant_array = model['constant'].values
d_t_array = target1['d_t'].values

In [158]:
#tranpose array
transposed_dt = np.transpose(d_t_array)
transposed_dt.shape

(558,)

In [149]:
w = cp.Variable(558)

obj = cp.Minimize(cp.norm(constant_array - transposed_dt @ w, 2))
constraints = [w <= 0.3, cp.sum(w) == 1]
problem = cp.Problem(obj, constraints)
result = problem.solve(verbose=True,solver=cp.SCS) ####scs is a first-order solver that uses an iterative splitting technique when just solving the whole thing is too expensive
#print("Optimal value:", result)
#print("Optimal weights:", w.value.round(decimals=2))
#print("Optimal value", problem.solve)
#print(w.value.round(decimals=2))
#print("Optimal value", problem.solve(verbose=True))

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Apr 14 08:58:57 AM: Your problem has 558 variables, 2 constraints, and 0 parameters.
(CVXPY) Apr 14 08:58:57 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 14 08:58:57 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 14 08:58:57 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 14 08:58:57 AM: Compiling problem (target solver=SCS).
(CVXPY) Apr 14 08:58:57 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

In [142]:
optimal_weights_df = pd.DataFrame(w.value.round(decimals=2), columns=["weights"])
print(optimal_weights_df)

     weights
0      -0.35
1      -0.35
2      -0.35
3      -0.35
4      -0.35
..       ...
553     0.16
554     0.16
555     0.17
556     0.19
557     0.21

[558 rows x 1 columns]


In [154]:
#joining dataframes
target1 = target1.dropna()
print(target1)

     sec_id security_identifier     quantity  market_price  \
2     88319           46649CAE3   44494.9284    101.102310   
3    195277           69374XAC4   28304.4240    101.104790   
4    196078           69375BAC1   65446.2652    101.782910   
12   206798           91282CAB7  492000.0000     99.578125   
14   207178           91282CAE1  243000.0000     97.406250   
..      ...                 ...          ...           ...   
548  206970           11135FAS0   31000.0000    117.454455   
549   77637           009090AA9  139639.0261     98.087030   
550   79179           84858EAA4  202525.2726     93.863960   
553   71556           90346WAA1       0.1551     91.503200   
556  194316           674599DG7  421000.0000     45.750000   

     market_value_plus_ai          sec_description1  \
2                45180.06           JPMMT 2018-4 A5   
3                28740.96            PSMC 2019-2 A3   
4                66899.43            PSMC 2019-3 A3   
12              490385.50          

In [156]:
result_df = pd.concat([optimal_weights_df, target1], axis=1)
pd.set_option("display.max_rows", None)
result_df = result_df.dropna()
# Print the result
#print(result_df)

     weights    sec_id security_identifier     quantity  market_price  \
2      -0.35   88319.0           46649CAE3   44494.9284    101.102310   
3      -0.35  195277.0           69374XAC4   28304.4240    101.104790   
4      -0.35  196078.0           69375BAC1   65446.2652    101.782910   
12     -0.35  206798.0           91282CAB7  492000.0000     99.578125   
14     -0.35  207178.0           91282CAE1  243000.0000     97.406250   
18     -0.35   89873.0           96328DBF0   22972.8616    100.351710   
21      0.05   89079.0           02005AGU6  140000.0000    101.251560   
26      0.12   90160.0           46648UAE4   31377.1172    100.304690   
27      0.13  192307.0           38013TAD3  100000.0000    100.819730   
28      0.14   86740.0           33850TAE8   10947.7286    100.077740   
39      0.20  195282.0           36258MAE4  100000.0000    102.973930   
41      0.20   92044.0           65341KBG0  104000.0000    103.245380   
44      0.22   91341.0           37045XCP9  173000.